<a href="https://colab.research.google.com/github/rukmals/ColabProject/blob/master/android_malware.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
import pandas as pd
import numpy as np




In [0]:
malware = pd.read_csv('permission.csv')

In [22]:
data_in = malware.columns.drop(['android','type'])
data_in
#malware[data_in].iloc[1].shape
X = []
for i in range(398):
    X.append(np.array(malware[data_in].iloc[i]))
    #np.concatenate((X, np.array(malware[data_in].iloc[i])))

print(np.shape(X))

(398, 329)


In [0]:
def layer_sizes(X, Y):
   
    n_x = np.shape(X)[0]
    n_h = 128
    n_y = np.shape(Y)[0]

    return (n_x, n_h, n_y)

In [30]:
Y = np.array([malware['type'][:329]])
print(np.shape(Y))

(1, 329)


In [39]:
layer_sizes(X, Y)

(398, 128, 1)

In [0]:
def initialize_parameters(n_x, n_h, n_y):
   
    W1 = np.random.randn(n_h,n_x)*0.01
    b1 = np.zeros((n_h,1))
    W2 = np.random.randn(n_y,n_h)*0.01
    b2 = np.zeros((n_y,1))
    assert (W1.shape == (n_h, n_x))
    assert (b1.shape == (n_h, 1))
    assert (W2.shape == (n_y, n_h))
    assert (b2.shape == (n_y, 1))
    
    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2}
    
    return parameters

In [41]:
layer = layer_sizes(X, Y)
para = initialize_parameters(layer[0],layer[1],layer[2])
para["W1"].shape
#para["b1"].shape

(128, 398)

In [0]:
def sigmoid(x):
    z = 1/(1 + np.exp(-x))
    return z

def forward_propagation(X, parameters):

    W1 = parameters["W1"]
    b1 = parameters["b1"]
    W2 = parameters["W2"]
    b2 = parameters["b2"]

    Z1 = np.dot(W1,X)+b1
    A1 = np.tanh(Z1)
    Z2 = np.dot(W2,A1)+b2
    A2 = sigmoid(Z2)
    assert(np.shape(A2) == (1, np.shape(X)[1]))
    
    cache = {"Z1": Z1,
             "A1": A1,
             "Z2": Z2,
             "A2": A2}
    
    return A2, cache


In [43]:
forward_propagation(X, para)
A2, cache = forward_propagation(X, para)
print(np.shape(A2))


(1, 329)


In [0]:
def compute_cost(A2, Y, parameters):
    
    
    m = Y.shape[1] # number of example

   
    cost = np.float64((-1.0/m) * (np.dot(Y, np.log(A2).T) + np.dot(1-Y, np.log(1-A2).T)))
    

    
    cost = np.squeeze(cost)
    cost = cost.astype(float) # makes sure cost is the dimension we expect. 
                                    # E.g., turns [[17]] into 17 
    assert(isinstance(cost, float))
    
    return cost

In [45]:
compute_cost(A2, Y, para)

0.6931426259046516

In [0]:
def backward_propagation(parameters, cache, X, Y):
   
    m = np.shape(X)[1]
    
    W1 = parameters["W1"]
    W2 = parameters["W2"]
    
 
    
    A1 = cache["A1"]
    A2 = cache["A2"]
    
   
   
    dZ2 = A2 - Y
    dW2 = (1/m)*np.dot(dZ2,A1.T)
    db2 = np.sum(dZ2,axis = 1,keepdims=True)/m
    dZ1 = np.dot(W2.T,dZ2) * (1 - np.power(A1,2))       
    dW1 = np.dot(dZ1,np.array(X).T)/m                     
    db1 = np.sum(dZ1, axis=1, keepdims=True)/m
    grads = {"dW1": dW1,
             "db1": db1,
             "dW2": dW2,
             "db2": db2}
    
    return grads
    
    


In [0]:
def update_parameters(parameters, grads, learning_rate = 1.2):
  
    W1 = parameters["W1"]
    b1 = parameters["b1"]
    W2 = parameters["W2"]
    b2 = parameters["b2"]
   
    dW1 = grads['dW1']
    db1 = grads['db1']
    dW2 = grads['dW2']
    db2 = grads['db2']
    
    W1 = W1 - learning_rate * dW1
    b1 = b1 - learning_rate * db1
    W2 = W2 - learning_rate * dW2
    b2 = b2 - learning_rate * db2
    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2}
    
    return parameters

In [0]:


def nn_model(X, Y, n_h, num_iterations = 10000, print_cost=False):
   
    
    np.random.seed(3)
    n_x = layer_sizes(X, Y)[0]
    n_y = layer_sizes(X, Y)[2]
    
    
    parameters = None
    
 
    parameters = initialize_parameters(n_x, n_h, n_y)
    W1 = parameters["W1"]
    b1 = parameters["b1"]
    W2 = parameters["W2"]
    b2 = parameters["b2"]
    # Loop (gradient descent)

    for i in range(0, num_iterations):
         
       
        A2 , cache = forward_propagation(X, parameters)
        
        cost = compute_cost(A2, Y, parameters)
        
        grads = backward_propagation(parameters, cache, X, Y)
        
        parameters = update_parameters(parameters, grads)
        
        if print_cost and i % 1000 == 0:
            print ("Cost after iteration %i: %f" %(i, cost))

    return parameters

In [0]:

def predict(parameters, X):
   
    A2,cache  = forward_propagation(X,parameters)
    predictions = (A2>0.5)
    
    
    return predictions

In [50]:
parameters = nn_model(X, Y, 128, num_iterations=10000, print_cost=True)
predictions = predict(parameters, X)
print ('Accuracy: %d' % float((np.dot(Y,predictions.T) + np.dot(1-Y,1-predictions.T))/float(Y.size)*100) + '%')

Cost after iteration 0: 0.693148
Cost after iteration 1000: 0.530532
Cost after iteration 2000: 0.525062
Cost after iteration 3000: 0.522965
Cost after iteration 4000: 0.521845
Cost after iteration 5000: 0.521142
Cost after iteration 6000: 0.520655
Cost after iteration 7000: 0.520296
Cost after iteration 8000: 0.519987
Cost after iteration 9000: 0.519175
Accuracy: 69%
